In [ ]:
lht_sensors = {
    'LHT65013': (62.234563, 25.672774),
    'LHT65010': (62.260777, 25.693876),
    'LHT65009': (62.222971, 25.804673),
    'LHT65008': (62.227604, 25.736853),
    'LHT65007': (62.286678, 25.74533),
    'LHT65006': (62.265198, 25.89008),
    'LHT65005': (62.197614, 25.720489),}

ws100_sensors = {
    'Saaritie':        (62.136788, 25.762473),
    'Tuulimyllyntie':  (62.221789, 25.695931),
    'Tähtiniementie':  (62.011127, 25.552755),
    'Kaakkovuori': (62.294362, 25.800196),
    'Kotaniementie':   (62.265705, 25.909542),}

geolocator = Nominatim(user_agent="sensor_map")
reverse_geocode = RateLimiter(geolocator.reverse, min_delay_seconds=1, swallow_exceptions=True)


def extract_street(address_dict: dict) -> str:
    # Try common street keys; fall back to suburb/city if needed
    for key in ['road', 'pedestrian', 'footway', 'cycleway', 'path', 'residential']:
        if key in address_dict:
            return address_dict[key]
    for fallback in ['neighbourhood', 'suburb', 'hamlet', 'village', 'town', 'city', 'county']:
        if fallback in address_dict:
            return address_dict[fallback]
    return "Unknown street"

# Optional: simple in-memory cache to avoid re-querying the same coordinates repeatedly
geocode_cache = {}

def get_street_name(lat: float, lon: float, language: str = 'en') -> str:
    key = (round(lat, 6), round(lon, 6), language)
    if key in geocode_cache:
        return geocode_cache[key]
    location = reverse_geocode((lat, lon), language=language)  
    if location and hasattr(location, "raw"):
        street = extract_street(location.raw.get('address', {}))
    else:
        street = "Unknown street"
    geocode_cache[key] = street
    return street


map_center = [62.24, 25.75]
sensors_map = folium.Map(location=map_center, zoom_start=11, tiles="OpenStreetMap")

lht_layer = folium.FeatureGroup(name="LHT Sensors", show=True)
ws100_layer = folium.FeatureGroup(name="WS100 Sensors", show=True)


for sensor_name, (lat, lon) in lht_sensors.items():
    street_name = get_street_name(lat, lon, language='en')  
    popup_html = f"<b>{sensor_name}</b><br>{street_name}<br>({lat:.6f}, {lon:.6f})"
    tooltip_text = f"{sensor_name} – {street_name}"
    folium.Marker(
        location=[lat, lon],
        popup=popup_html,
        tooltip=tooltip_text,
        icon=folium.Icon(color="blue", icon="info-sign"),
    ).add_to(lht_layer)


for site_name, (lat, lon) in ws100_sensors.items():
    street_name = get_street_name(lat, lon) 
    popup_html = f"<b>{site_name} (WS100)</b><br>{street_name}<br>({lat:.6f}, {lon:.6f})"
    tooltip_text = f"{site_name} – {street_name}"
    folium.Marker(
        location=[lat, lon],
        popup=popup_html,
        tooltip=tooltip_text,
        icon=folium.Icon(color="red", icon="info-sign"),).add_to(ws100_layer)

lht_layer.add_to(sensors_map)
ws100_layer.add_to(sensors_map)
folium.LayerControl(collapsed=False).add_to(sensors_map)

sensors_map


In [ ]:


lht_sensor = {
    'Keltimaentie-LHT65013': (62.234563, 25.672774),
    'Hikipolku-LHT65010': (62.260777, 25.693876),
    'Hameenpohjantie-LHT65009': (62.222971, 25.804673),
    'Survontie-LHT65008': (62.227604, 25.736853),
    'Ritopohantie-LHT65007': (62.286678, 25.74533),
    'Kaunisharjuntie-LHT65006': (62.265198, 25.89008),
    'Keilonkankaantie-LHT65005': (62.197614, 25.720489),}


def haversine(lat1, lon1, lat2, lon2):
    R = 6371  
    dlat = np.radians(lat2 - lat1)
    dlon = np.radians(lon2 - lon1)
    a = np.sin(dlat/2)**2 + np.cos(np.radians(lat1)) * np.cos(np.radians(lat2)) * np.sin(dlon/2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
    return R * c

distances = pd.DataFrame(index=lht_sensor.keys(), columns=ws100_sensors .keys())
for lht, (lat1, lon1) in lht_sensor.items():
    for ws, (lat2, lon2) in ws100_sensors .items():
        distances.loc[lht, ws] = haversine(lat1, lon1, lat2, lon2)

print("Distance Matrix (km):\n", distances)
nearest_pairs = distances.idxmin(axis=1)
print("\nNearest Pairs:\n", nearest_pairs)